In [1]:
from fastai.text import *
import json
import html
import re
import pickle
from collections import Counter
import random
import pandas as pd
import numpy as np
from pathlib import Path
import sklearn
from sklearn import model_selection
from functools import partial
from collections import Counter, defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.utils 
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import dataset, dataloader
import torch.optim as optim
import torch.nn.functional as F

import time
import math
import sys
import data

snli_root = './data/SNLI/'

In [ ]:
#! wget https://github.com/briandw/SiameseULMFiT/releases/download/1/data.zip
#! unzip ./data.zip

In [16]:
# load and process the all the sentences, just to get the LM trained
raw_text = []
for file in [f"{snli_root}/snli_train.json", f"{snli_root}/snli_dev.json", f"{snli_root}/snli_test.json"]:
    with open(file) as fp:
        content = json.load(fp)
        for item in content:
            raw_text.append(item[0])
            raw_text.append(item[1])
print(len(raw_text))

1139636


In [17]:
#split the language model data into train and validation sets
lm_train, lm_valid = sklearn.model_selection.train_test_split(raw_text, test_size=0.1)
df_trn = pd.DataFrame(lm_train)
df_val = pd.DataFrame(lm_valid)

In [18]:
BOS = 'x_bos'  # beginning-of-sentence tag

re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def get_texts(df):
    texts = df[0].astype(str)
    texts = list(texts.apply(fixup).values)
    texts = f'{BOS} ' + df[0].astype(str)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok

In [19]:
tok_trn = np.concatenate(get_texts(df_trn))
tok_val = np.concatenate(get_texts(df_val))

In [20]:
tok_val[:100]

array(['x_bos', 'a', 'woman', 'in', 'a', 'red', 'coat', 'is', 'leaving', 'a', 'photo', 'shop', '.', 'x_bos',
       'a', 'young', 'redheaded', 'woman', 'laughs', 'at', 'something', '.', 'x_bos', 'a', 'man', 'comes',
       'out', 'of', 'the', 'gene', 'pool', 'after', 'swimming', '.', 'x_bos', 'the', 'women', 'are', 'in',
       'a', 'play', '.', 'x_bos', 'seven', 'men', 'in', 'orange', 'work', 'at', 'night', 'on', 'a',
       'railway', ',', 'next', 'to', 'a', 'train', '.', 'x_bos', 'a', 'boy', 'is', 'playing', 'in', 'the',
       'sand', '.', 'x_bos', 'people', 'are', 'walking', 'outside', '.', 'x_bos', 'a', 'man', 'is', 'seated',
       'in', 'front', 'of', 'a', 'big', 'calvin', 'klein', 'ad', '.', 'x_bos', 'people', 'wearing', 'white',
       'helmets', ',', 'looking', 'into', 'a', 'crowded', 'street', 'corner'], dtype='<U24')

In [21]:
#save our work
np.save(f'{snli_root}tok_trn.npy', tok_trn)
np.save(f'{snli_root}tok_val.npy', tok_val)

In [22]:
tok_trn = np.load(f'{snli_root}tok_trn.npy')
tok_val = np.load(f'{snli_root}tok_val.npy')

In [23]:
freq = Counter(np.concatenate([tok_trn, tok_val]))
freq.most_common(25)

[('a', 1495314),
 ('x_bos', 1139636),
 ('.', 999009),
 ('the', 554918),
 ('in', 423676),
 ('is', 387678),
 ('man', 276610),
 ('on', 245026),
 ('and', 215080),
 ('are', 206697),
 ('of', 200366),
 ('with', 176065),
 ('woman', 143029),
 ('two', 126872),
 ('people', 125577),
 (',', 119825),
 ('to', 118657),
 ('at', 102392),
 ('wearing', 84371),
 ('an', 83393),
 ('his', 75518),
 ('shirt', 65434),
 ('young', 64082),
 ('men', 63368),
 ('playing', 61542)]

In [25]:
len(freq)

34153

In [26]:
max_vocab = 60000
min_freq = 1
itos = [o for o, c in freq.most_common(max_vocab) if c>=min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')
stoi = defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

In [27]:
len(stoi)

34155

In [28]:
trn_lm = np.array([stoi[p] for p in tok_trn])
val_lm = np.array([stoi[p] for p in tok_val])

In [29]:
#save results
pickle.dump(itos, open(f'{snli_root}itos.pkl', 'wb'))
np.save(f'{snli_root}trn_lm.npy', trn_lm)
np.save(f'{snli_root}val_lm.npy', val_lm)

In [30]:
#load the results so we can pick it up from here 
itos = pickle.load(open(f'{snli_root}itos.pkl', 'rb'))
trn_lm = np.load(f'{snli_root}trn_lm.npy')
val_lm = np.load(f'{snli_root}val_lm.npy')

stoi = defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
vocab_size = len(itos)
vocab_size

34155

In [31]:
for word in val_lm[:100]:
    print(itos[word], end=" ")

x_bos a woman in a red coat is leaving a photo shop . x_bos a young redheaded woman laughs at something . x_bos a man comes out of the gene pool after swimming . x_bos the women are in a play . x_bos seven men in orange work at night on a railway , next to a train . x_bos a boy is playing in the sand . x_bos people are walking outside . x_bos a man is seated in front of a big calvin klein ad . x_bos people wearing white helmets , looking into a crowded street corner 